# Spam Detector

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Retrieve the Data

The data is located at [https://static.bc-edx.com/mbc/ai/m4/datasets/spam-data.csv](https://static.bc-edx.com/mbc/ai/m4/datasets/spam-data.csv)

Dataset Source: [UCI Machine Learning Library](https://archive-beta.ics.uci.edu/dataset/94/spambase)

Import the data using Pandas. Display the resulting DataFrame to confirm the import was successful.

In [2]:
# Import the data
data = pd.read_csv("https://static.bc-edx.com/mbc/ai/m4/datasets/spam-data.csv")
print(data.shape)
data.info()

(4601, 58)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4601 entries, 0 to 4600
Data columns (total 58 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   word_freq_make              4601 non-null   float64
 1   word_freq_address           4601 non-null   float64
 2   word_freq_all               4601 non-null   float64
 3   word_freq_3d                4601 non-null   float64
 4   word_freq_our               4601 non-null   float64
 5   word_freq_over              4601 non-null   float64
 6   word_freq_remove            4601 non-null   float64
 7   word_freq_internet          4601 non-null   float64
 8   word_freq_order             4601 non-null   float64
 9   word_freq_mail              4601 non-null   float64
 10  word_freq_receive           4601 non-null   float64
 11  word_freq_will              4601 non-null   float64
 12  word_freq_people            4601 non-null   float64
 13  word_freq_report      

In [3]:
if data.isna().values.any():
  print('There are missing values in the dataset.')
else:
  print('There are no missing (null) values.')
duplicated_rows = data[data.duplicated()]
if len(duplicated_rows) > 0:
  print(f'There are {len(duplicated_rows)} duplicated rows in the dataset; perhaps by-product of sample data generation?')
print('All values are numerical.')
data.sample(7)

There are no missing (null) values.
There are 391 duplicated rows in the dataset; perhaps by-product of sample data generation?
All values are numerical.


,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
1571,0.00,0.00,0.00,0.0,0.68,0.00,0.00,0.00,0.00,1.81,...,0.159,0.558,0.159,0.199,0.000,0.000,6.091,83,530,1
995,0.00,0.41,0.41,0.0,2.06,0.00,1.65,0.82,0.00,0.00,...,0.000,0.265,0.000,0.199,0.000,0.000,15.892,226,445,1
1639,0.87,0.17,0.52,0.0,0.00,0.32,0.00,0.04,0.29,0.42,...,0.016,0.058,0.000,0.639,0.165,0.182,3.697,117,3498,1
2794,0.00,0.00,1.33,0.0,0.00,0.00,0.00,0.00,0.66,0.00,...,0.300,0.200,0.100,0.000,0.000,0.000,1.979,12,97,0
1798,0.76,0.00,0.38,0.0,0.12,0.25,0.00,0.12,0.12,0.00,...,0.000,0.022,0.000,0.661,0.088,0.000,2.256,21,325,1
710,0.40,0.14,0.32,0.0,0.25,0.18,0.03,1.01,0.40,0.40,...,0.012,0.042,0.073,0.343,0.141,0.000,3.328,181,1621,1
666,0.00,0.00,0.84,0.0,0.56,0.00,0.00,0.56,0.00,0.00,...,0.000,0.278,0.000,0.046,0.000,0.000,1.661,6,118,1


## Predict Model Performance

You will be creating and comparing two models on this data: a Logistic Regression, and a Random Forests Classifier. Before you create, fit, and score the models, make a prediction as to which model you think will perform better. You do not need to be correct!

Write down your prediction in the designated cells in your Jupyter Notebook, and provide justification for your educated guess.

*I predict that the Random Forest classifier will perform better since the dataset is quite complex, with a large number of features, and there is a risk that the self tuning approach of the Logistic Regression model as it works toward the optimal model (sharpens the S-curve) could result in overfitting. The ensemble learning aspects of the Random Forest Classifier should mitigate the risk of overfitting by taking a majority vote from a multitude of predictions by models based on random sub-sets of the features (less complex dataset).*

## Split the Data into Training and Testing Sets

In [4]:
# Create the labels set `y` and features DataFrame `X`
y = data['spam']
X = data.copy()
X = X.drop(columns='spam')

In [5]:
# Check the balance of the labels variable (`y`) by using the `value_counts` function.
y.value_counts()

0    2788
1    1813
Name: spam, dtype: int64

In [6]:
# Split the data into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total
4576,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.131,0.000,0.0,0.000,0.0,1.488,5,64
4401,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.000,0.000,0.0,0.000,0.0,1.571,5,11
3707,0.17,0.0,0.17,0.0,0.00,0.0,0.0,0.0,0.8,0.0,...,0.0,0.253,0.168,0.084,0.0,0.024,0.0,4.665,81,1031
2362,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.000,0.000,0.0,0.000,0.0,4.228,53,148
1537,0.00,0.0,0.00,0.0,2.17,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.000,0.000,0.0,0.000,0.0,1.333,5,16


## Scale the Features

Use the `StandardScaler` to scale the features data. Remember that only `X_train` and `X_test` DataFrames should be scaled.

In [7]:
# from sklearn.preprocessing import StandardScaler (imported at top)

# Create the StandardScaler instance
scaler = StandardScaler()

In [8]:
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)

In [9]:
# Scale the training (and testing) data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled

array([[-0.35811925, -0.16744248, -0.56071981, ..., -0.13974678,
        -0.23023328, -0.37548424],
       [-0.35811925, -0.16744248, -0.56071981, ..., -0.13640572,
        -0.23023328, -0.46466704],
       [ 0.21317665, -0.16744248, -0.22391235, ..., -0.01186062,
         0.13008597,  1.25168127],
       ...,
       [-0.35811925, -0.16744248,  0.94500764, ..., -0.10138496,
        -0.16385868, -0.06418577],
       [-0.35811925, -0.16744248,  1.16294188, ..., -0.1351981 ,
        -0.20178702, -0.01707033],
       [-0.35811925, -0.16744248, -0.56071981, ..., -0.15524447,
        -0.23971537, -0.42933046]])

## Create and Fit a Logistic Regression Model

Create a Logistic Regression model, fit it to the training data, make predictions with the testing data, and print the model's accuracy score. You may choose any starting settings you like.

In [10]:
# Train a Logistic Regression model and print the model score
# from sklearn.linear_model import LogisticRegression (imported at top)
lr_model = LogisticRegression(random_state=1)
lr_model.fit(X_train_scaled, y_train)
print(f"Log Reg. Training Data Score: {lr_model.score(X_train_scaled, y_train)}")
print(f"Log Reg. Testing Data Score: {lr_model.score(X_test_scaled, y_test)}")

Log Reg. Training Data Score: 0.9298550724637681
Log Reg. Testing Data Score: 0.9287576020851434


In [11]:
# Make and save testing predictions with the saved logistic regression model using the test data
lr_predictions = lr_model.predict(X_test_scaled)
# Review the predictions
lr_predictions

array([0, 0, 1, ..., 0, 0, 1])

In [12]:
# Calculate the accuracy score by evaluating `y_test` vs. `testing_predictions`.
lr_accuracy = accuracy_score(y_test, lr_predictions)
print(f'Logistic Regression model accuracy score: {lr_accuracy * 100:.3f}%')

Logistic Regression model accuracy score: 92.876%


## Create and Fit a Random Forest Classifier Model

Create a Random Forest Classifier model, fit it to the training data, make predictions with the testing data, and print the model's accuracy score. You may choose any starting settings you like.

In [13]:
# Train a Random Forest Classifier model and print the model score
# from sklearn.ensemble import RandomForestClassifier (imported at top)
rf_model = RandomForestClassifier(n_estimators=73, random_state=1)
rf_model.fit(X_train_scaled, y_train)
print(f"Rnd Frst. Training Data Score: {rf_model.score(X_train_scaled, y_train)}")
print(f"Rnd Frst. Testing Data Score: {rf_model.score(X_test_scaled, y_test)}")

Rnd Frst. Training Data Score: 0.9997101449275362
Rnd Frst. Testing Data Score: 0.9669852302345786


In [14]:
# Make and save testing predictions with the saved logistic regression model using the test data
rf_predictions = rf_model.predict(X_test_scaled)

# Review the predictions
rf_predictions

array([1, 1, 1, ..., 1, 0, 1])

In [15]:
# Calculate the accuracy score by evaluating `y_test` vs. `testing_predictions`.
rf_accuracy = accuracy_score(y_test, rf_predictions)
print(f'Random Forest Classifier model accuracy score: {rf_accuracy * 100:.3f}%')

Random Forest Classifier model accuracy score: 96.699%


In [16]:
improve = (rf_accuracy - lr_accuracy) * 100
print(f'Comparative model improvement in accuracy scores was {improve:.3f}%')

Comparative model improvement in accuracy scores was 3.823%


## Evaluate the Models

Which model performed better? How does that compare to your prediction? Write down your results and thoughts in the following markdown cell.

*The Random Forest Model performed better as predicted however the improvement was more significant against the training data (up 7% to a near perfect result). The improvement in test data scores (prediction accuracy) was lower, 3.8%, taking it to 96.7% however both the improvement and the result here are statistically significant enough to rate it a better model overall than the one delivered by the Logistic Regression model.*

*Model Argument Impact:*
*I experimented with the LogisticRegression() max_iter argument but there was no significant impact on accuracy. By contrast with the RandomForestClassifier(), the n_estimators value of 73 seemed to provide the most significant improvement in test data accuracy score. Similarly, adjusting the random_state variable on the train_test_split() function resulted in marginal improvements in the fitting process (and results for both models).*

*As an additional note, I had suggested that there was a greater risk of over-fitting with the Logistic Regression model (than the Random Forest one) however the train and test scores were very close with the Logistic Regression model so I do not suspect it occurred to any significance. Rather, given the relative scores of both models is it more likely in this case that the Random Forest Classifier is simply a better performing model for the complexity of this dataset.*